In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pathlib import Path

from secrets import *

# Set Chrome to download PDFs instead of viewing them in the browser
chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # This will force PDFs to be downloaded
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)

wait = WebDriverWait(driver, 10)

driver.get('https://wellsfargo.com')

userid_box = driver.find_element('id', 'userid')
userid_box.send_keys(wellsfargo_user)

# Apparently necessary so the site doesn't suspect a bot
time.sleep(2)
password_box = driver.find_element('id', 'password')

# TODO get this from a file - there's probably a package like .dotenv or something like it
password_box.send_keys(wellsfargo_password)
password_box.send_keys(Keys.RETURN)

account = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Accounts')]")))
account.click()

statements = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "View Statements & Documents")))
statements.click()

statements_and_disclosures = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Statements and Disclosures')]")))
statements_and_disclosures.click()

no_selection = True
account_label = 'Select account'
select_account = None
for account in wellsfargo_accounts:

    wait = WebDriverWait(driver, 10)
    if no_selection:
        select_account = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{account_label}')]")))
        no_selection = False
    else:
        select_account = wait.until(EC.element_to_be_clickable((By.XPATH, f"//label[contains(text(), '{account_label}')]")))

    # select_account.click() doesn't work here, but this does
    driver.execute_script("arguments[0].click();", select_account)

    print(f'Downloading statements for {account}...')
    account_item = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{account}')]")))
    account_item.click()

    statement_item = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Statement ')]")))
    statement_items = driver.find_elements(By.XPATH, "//span[contains(text(), 'Statement ')]")
    for item in statement_items:
        item.click()
        
    account_label = account